In [221]:
import scipy.stats as st
from scipy.stats import poisson
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [222]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [223]:
df.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [224]:
df = df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [225]:
df.describe()

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
v1    5572 non-null object
v2    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [227]:
df = pd.get_dummies(df, columns=['v1']).drop(columns=['v1_ham'])

In [228]:
x_bar = df.v1_spam.sum()/5572
x_bar

0.13406317300789664

In [229]:
rv = poisson(x_bar)
rv

In [230]:
h0 = .125
n = len(df)
s = df.v1_spam.std(ddof=1)
se = s/np.sqrt(n)

In [231]:
z_value = (x_bar - h0)/se
z_value

1.985404480388316

In [232]:
st.zscore(df.v1_spam)

array([-0.39347002, -0.39347002,  2.54148972, ..., -0.39347002,
       -0.39347002, -0.39347002])

In [233]:
1 -st.norm.cdf(z_value) # Part 2 - Reject the null hypothesis

0.023549743949760393

In [234]:
lam = df.v1_spam.sum()/28
lam

26.678571428571427

In [235]:
rv = poisson(lam)
1 - rv.cdf(30) # probability of >= 30 emails/day

0.2251758885540568

In [236]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [237]:
df = df.rename(columns= {'v2': 'Message', 'v1_spam':'Spam'})

In [238]:
counts = count_vect.fit_transform(df.Message)
tfidfs = tfidf_transformer.fit_transform(counts)
counts

<5572x8672 sparse matrix of type '<class 'numpy.int64'>'
	with 73916 stored elements in Compressed Sparse Row format>

In [239]:
X_train, X_test, y_train, y_test = train_test_split(tfidfs, df.Spam, test_size=0.33, random_state=42)
X_train

<3733x8672 sparse matrix of type '<class 'numpy.float64'>'
	with 49296 stored elements in Compressed Sparse Row format>

In [240]:
nb = MultinomialNB().fit(X_train, y_train)
predictions = nb.predict(X_test)
predictions.sum()

167

In [241]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [242]:
y_test.sum()

252

In [243]:
167/252

0.6626984126984127

In [244]:
y_df = y_test.to_frame()

In [245]:
y_df.insert(column='Prediction', value = predictions, loc=1)

In [246]:
y_df.head()

,Spam,Prediction
3245,0,0
944,0,0
1044,1,0
2484,0,0
812,1,1


In [247]:
y_df = y_df.rename(columns= {'Spam': 'Actual'})
y_df.head()
type(y_df)

pandas.core.frame.DataFrame

In [253]:
FP = y_df.Prediction(['Prediction'] > 0 & ['Actual'] < 1)

TypeError: unsupported operand type(s) for &: 'int' and 'list'

In [ ]:
FP.count()